In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# load source datasets containing finer granular data
# We only need the geometry of one source file, as the source datasets contain the
# column blknr (block number) which identifies each row (block) uniquely.
df1 = gpd.read_file('raw/fernwaerme.geojson')
df2 = gpd.read_file('raw/gas.geojson').drop(columns='geometry')
df3 = gpd.read_file('raw/strom.geojson').drop(columns='geometry')
# Inefficient, but only done once anyway...

# we want to find the respective plr for each source row, so we load the plr dataset
plr = gpd.read_file('../plr/plr_only.geojson')

# plr without geometry (for merging later on)
plr_no_geo = plr.drop(columns='geometry').sort_values(by='PLR_ID')

# check for same crs (coordinate reference system)
df1.crs == plr.crs

True

In [3]:
display(df1.shape, df2.shape, df3.shape, plr.shape)
display(df1.head(), df2.head(), df3.head(), plr.head())

(16390, 5)

(16390, 7)

(16390, 7)

(542, 3)

,blknr,verbrauch_mwh,verbrauch_mwh2020,verbrauch_mwh2021,geometry
0,001001,NaN,NaN,NaN,"MULTIPOLYGON (((386738.147 5822363.306, 386786..."
1,001002,NaN,NaN,NaN,"MULTIPOLYGON (((388156.019 5822081.011, 388241..."
2,001007,2568.032,2684.242,2786.803,"MULTIPOLYGON (((386436.837 5821594.107, 386599..."
3,001010,NaN,NaN,NaN,"MULTIPOLYGON (((388575.857 5822156.894, 388492..."
4,001012,1305.403,1444.354,1432.731,"MULTIPOLYGON (((385669.028 5821351.571, 385680..."


,blknr,verbrauch_mwh,ausgeschlossen,verbrauch_mwh2020,ausgeschlossen2020,verbrauch_mwh2021,ausgeschlossen2021
0,001001,NaN,Keine Angabe zum Verbrauch verfügbar,NaN,Keine Angabe zum Verbrauch verfügbar,NaN,Keine Angabe zum Verbrauch verfügbar
1,001002,NaN,Der Verbrauch wird aus Datenschutzgründen nich...,NaN,Der Verbrauch wird aus Datenschutzgründen nich...,NaN,Der Verbrauch wird aus Datenschutzgründen nich...
2,001007,4426.624,NaN,4521.800,NaN,5140.985,NaN
3,001010,NaN,Der Verbrauch wird aus Datenschutzgründen nich...,184.973,NaN,NaN,Der Verbrauch wird aus Datenschutzgründen nich...
4,001012,2696.962,NaN,2897.430,NaN,3202.943,NaN


,blknr,verbrauch_mwh,ausgeschlossen,verbrauch_mwh2020,ausgeschlossen2020,verbrauch_mwh2021,ausgeschlossen2021
0,001001,35.002,NaN,44.962,NaN,38.291,NaN
1,001002,14178.253,NaN,NaN,Der Verbrauch wird aus Datenschutzgründen nich...,NaN,Der Verbrauch wird aus Datenschutzgründen nich...
2,001007,4371.976,NaN,5887.394,NaN,4609.599,NaN
3,001010,299.238,NaN,304.234,NaN,301.856,NaN
4,001012,1093.980,NaN,1434.601,NaN,1123.465,NaN


,PLR_ID,PLR_NAME,geometry
0,11501341,Karlshorst Süd,"POLYGON ((399848.954 5815619.506, 399836.521 5..."
1,03701659,Immanuelkirchstraße,"POLYGON ((393175.012 5821135.577, 393183.490 5..."
2,07601340,Tirschenreuther Ring Ost,"POLYGON ((388394.110 5807792.070, 388404.779 5..."
3,02500831,Wismarplatz,"POLYGON ((396010.552 5819339.570, 395991.806 5..."
4,12601134,Märkisches Zentrum,"POLYGON ((387367.428 5828619.763, 387389.202 5..."


In [4]:
display("df1", df1.columns,"df2", df2.columns,"df3", df3.columns)

'df1'

Index(['blknr', 'verbrauch_mwh', 'verbrauch_mwh2020', 'verbrauch_mwh2021',
       'geometry'],
      dtype='object')

'df2'

Index(['blknr', 'verbrauch_mwh', 'ausgeschlossen', 'verbrauch_mwh2020',
       'ausgeschlossen2020', 'verbrauch_mwh2021', 'ausgeschlossen2021'],
      dtype='object')

'df3'

Index(['blknr', 'verbrauch_mwh', 'ausgeschlossen', 'verbrauch_mwh2020',
       'ausgeschlossen2020', 'verbrauch_mwh2021', 'ausgeschlossen2021'],
      dtype='object')

In [5]:
df1.columns = ['blknr', 'district_heating_mwh_2022', 'district_heating_mwh_2020', 'district_heating_mwh_2021', 'geometry']
df1 = df1[['blknr', 'district_heating_mwh_2022', 'district_heating_mwh_2021', 'district_heating_mwh_2020', 'geometry']]

# Columns named ausgeschlossenYYYY are redundant and can be dropped, they imply
# nan values of their respective verbrauch_mwhYYYY columns
df2.columns = [
    'blknr', 'gas_mwh_2022', 'ausgeschlossen', 'gas_mwh_2020',
    'ausgeschlossen2020', 'gas_mwh_2021', 'ausgeschlossen2021']
df2 = df2[['blknr', 'gas_mwh_2022', 'gas_mwh_2021', 'gas_mwh_2020']]


df3.columns = [
    'blknr', 'electricity_mwh_2022', 'ausgeschlossen', 'electricity_mwh_2020',
    'ausgeschlossen2020', 'electricity_mwh_2021', 'ausgeschlossen2021']
df3 = df3[['blknr', 'electricity_mwh_2022', 'electricity_mwh_2021', 'electricity_mwh_2020']]

In [6]:
display(df1.head(), df2.head(), df3.head())

,blknr,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,geometry
0,001001,NaN,NaN,NaN,"MULTIPOLYGON (((386738.147 5822363.306, 386786..."
1,001002,NaN,NaN,NaN,"MULTIPOLYGON (((388156.019 5822081.011, 388241..."
2,001007,2568.032,2786.803,2684.242,"MULTIPOLYGON (((386436.837 5821594.107, 386599..."
3,001010,NaN,NaN,NaN,"MULTIPOLYGON (((388575.857 5822156.894, 388492..."
4,001012,1305.403,1432.731,1444.354,"MULTIPOLYGON (((385669.028 5821351.571, 385680..."


,blknr,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020
0,001001,NaN,NaN,NaN
1,001002,NaN,NaN,NaN
2,001007,4426.624,5140.985,4521.800
3,001010,NaN,NaN,184.973
4,001012,2696.962,3202.943,2897.430


,blknr,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020
0,001001,35.002,38.291,44.962
1,001002,14178.253,NaN,NaN
2,001007,4371.976,4609.599,5887.394
3,001010,299.238,301.856,304.234
4,001012,1093.980,1123.465,1434.601


In [7]:
# Merging the source datasets on blknr
df = df1 \
    .merge(df2, on='blknr', how='inner') \
    .merge(df3, on='blknr', how='inner')
display(df.shape, df.head())

(16390, 11)

,blknr,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,geometry,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020
0,001001,NaN,NaN,NaN,"MULTIPOLYGON (((386738.147 5822363.306, 386786...",NaN,NaN,NaN,35.002,38.291,44.962
1,001002,NaN,NaN,NaN,"MULTIPOLYGON (((388156.019 5822081.011, 388241...",NaN,NaN,NaN,14178.253,NaN,NaN
2,001007,2568.032,2786.803,2684.242,"MULTIPOLYGON (((386436.837 5821594.107, 386599...",4426.624,5140.985,4521.800,4371.976,4609.599,5887.394
3,001010,NaN,NaN,NaN,"MULTIPOLYGON (((388575.857 5822156.894, 388492...",NaN,NaN,184.973,299.238,301.856,304.234
4,001012,1305.403,1432.731,1444.354,"MULTIPOLYGON (((385669.028 5821351.571, 385680...",2696.962,3202.943,2897.430,1093.980,1123.465,1434.601


In [8]:
# nan percentage per column
nan_percentage = df.isna().mean().round(4) * 100
# print to markdown
print(nan_percentage.to_markdown())
nan_percentage.to_csv('energy_consumption_nan_percentage.csv', header=False)

|                           |     0 |
|:--------------------------|------:|
| blknr                     |  0    |
| district_heating_mwh_2022 | 85.19 |
| district_heating_mwh_2021 | 85.28 |
| district_heating_mwh_2020 | 85.52 |
| geometry                  |  0    |
| gas_mwh_2022              | 38.17 |
| gas_mwh_2021              | 34.51 |
| gas_mwh_2020              | 34.69 |
| electricity_mwh_2022      | 23.38 |
| electricity_mwh_2021      | 24.92 |
| electricity_mwh_2020      | 24.94 |


### Spatial join of block and plr geometry
- ``df_with_plr = gpd.sjoin(df, plr, how='inner', predicate='within')`` -> row count 16390 -> 12070 (not all blocks get assigned)
- --> Joining these geometries can be tricky as the blocks and plrs are polygons, but the finer coarse blocks don't necessarily share boarders with the plrs...
- So, let's try calculating every blocks centroid first and then have these points assigned to their respective plr polygon...


In [9]:
df.geometry = df.geometry.centroid

In [10]:
# Spatial join assigns each source polygon to its respective plr
df_with_plr = gpd.sjoin(df, plr, how='inner', predicate='within').drop(columns="geometry")

print("df shape:", df.shape, "\ndf_with_plr shape:", df_with_plr.shape)

df shape: (16390, 11) 
df_with_plr shape: (16385, 13)


- with this approach only 5 blocks are not assigned, this is ok for now
- `predicate`: withing/intersects makes no difference...

In [11]:
df_with_plr.sample(5)

,blknr,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020,index_right,PLR_ID,PLR_NAME
10509,124019,NaN,NaN,NaN,985.678,1193.090,1091.448,162.495,175.403,174.288,265,09100306,Späthsfelde
16301,194615,2268.705,2561.535,2053.301,NaN,NaN,NaN,1759.380,1777.348,1698.256,438,10200421,Helle Mitte
12374,145053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131,11501340,Karlshorst Nord
9661,107902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129,03601244,Humannplatz
6940,080078,NaN,NaN,NaN,994.996,1208.003,1044.679,267.578,294.911,256.271,328,08200726,Hufeisensiedlung


In [12]:
df_with_plr.columns

Index(['blknr', 'district_heating_mwh_2022', 'district_heating_mwh_2021',
       'district_heating_mwh_2020', 'gas_mwh_2022', 'gas_mwh_2021',
       'gas_mwh_2020', 'electricity_mwh_2022', 'electricity_mwh_2021',
       'electricity_mwh_2020', 'index_right', 'PLR_ID', 'PLR_NAME'],
      dtype='object')

In [13]:
# reordering columns, dropping index_right
df_with_plr = df_with_plr[[
    'blknr', 'PLR_ID', 'PLR_NAME',
    'electricity_mwh_2022', 'electricity_mwh_2021', 'electricity_mwh_2020',
    'district_heating_mwh_2022', 'district_heating_mwh_2021', 'district_heating_mwh_2020',
    'gas_mwh_2022', 'gas_mwh_2021', 'gas_mwh_2020']]
df_with_plr.head()

,blknr,PLR_ID,PLR_NAME,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020
0,001001,01200517,Huttenkiez,35.002,38.291,44.962,NaN,NaN,NaN,NaN,NaN,NaN
1,001002,01200517,Huttenkiez,14178.253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001007,01200517,Huttenkiez,4371.976,4609.599,5887.394,2568.032,2786.803,2684.242,4426.624,5140.985,4521.800
3,001010,01200517,Huttenkiez,299.238,301.856,304.234,NaN,NaN,NaN,NaN,NaN,184.973
4,001012,01200517,Huttenkiez,1093.980,1123.465,1434.601,1305.403,1432.731,1444.354,2696.962,3202.943,2897.430


In [14]:
df_with_plr.to_csv('energy_consumption_per_block.csv', index=False)

### PLR summary

In [15]:
# only keep summary relevant features, get rid of geometry -> regular pandas dataframe
feature_list = [
    'PLR_ID',
    'electricity_mwh_2022', 'electricity_mwh_2021', 'electricity_mwh_2020',
    'district_heating_mwh_2022', 'district_heating_mwh_2021', 'district_heating_mwh_2020',
    'gas_mwh_2022', 'gas_mwh_2021', 'gas_mwh_2020']
df_with_plr = df_with_plr[feature_list]

In [16]:
summary = df_with_plr.groupby('PLR_ID').sum().reset_index()
display(summary.shape, summary.head())

(542, 10)

,PLR_ID,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020
0,01100101,28062.831,25324.645,27031.742,47523.801,51727.970,45712.122,165.235,169.608,184.124
1,01100102,90600.670,83197.539,87521.181,136019.818,143989.539,121624.071,266.090,1668.815,1698.517
2,01100103,23410.741,22884.915,23435.485,29846.244,33420.418,28116.120,22650.193,25143.113,21310.274
3,01100104,29443.289,27290.775,27335.933,15878.454,16391.735,13366.927,16936.874,20560.798,19195.664
4,01100205,75180.474,67578.132,75715.642,75519.883,80323.656,66730.236,2271.953,2706.023,3005.112


In [17]:
# Now we only have the PLR_ID, in the final dataset we also want the PLR_NAME
summary = summary.merge(plr_no_geo, on='PLR_ID', how='inner')

# Reorder columns
cols_ordered = ["PLR_ID", "PLR_NAME"] + summary.columns[1:-1].tolist()
summary = summary[cols_ordered]

display(summary.shape, summary.head())

(542, 11)

,PLR_ID,PLR_NAME,electricity_mwh_2022,electricity_mwh_2021,electricity_mwh_2020,district_heating_mwh_2022,district_heating_mwh_2021,district_heating_mwh_2020,gas_mwh_2022,gas_mwh_2021,gas_mwh_2020
0,01100101,Stülerstraße,28062.831,25324.645,27031.742,47523.801,51727.970,45712.122,165.235,169.608,184.124
1,01100102,Großer Tiergarten,90600.670,83197.539,87521.181,136019.818,143989.539,121624.071,266.090,1668.815,1698.517
2,01100103,Lützowstraße,23410.741,22884.915,23435.485,29846.244,33420.418,28116.120,22650.193,25143.113,21310.274
3,01100104,Körnerstraße,29443.289,27290.775,27335.933,15878.454,16391.735,13366.927,16936.874,20560.798,19195.664
4,01100205,Wilhelmstraße,75180.474,67578.132,75715.642,75519.883,80323.656,66730.236,2271.953,2706.023,3005.112


In [18]:
summary.to_csv('energy_consumption_per_plr.csv', index=False)